In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "High-Density_Lipoprotein_Deficiency"
cohort = "GSE34945"

# Input paths
in_trait_dir = "../../input/GEO/High-Density_Lipoprotein_Deficiency"
in_cohort_dir = "../../input/GEO/High-Density_Lipoprotein_Deficiency/GSE34945"

# Output paths
out_data_file = "../../output/preprocess/High-Density_Lipoprotein_Deficiency/GSE34945.csv"
out_gene_data_file = "../../output/preprocess/High-Density_Lipoprotein_Deficiency/gene_data/GSE34945.csv"
out_clinical_data_file = "../../output/preprocess/High-Density_Lipoprotein_Deficiency/clinical_data/GSE34945.csv"
json_path = "../../output/preprocess/High-Density_Lipoprotein_Deficiency/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Candidate SNPs association with APOC3"
!Series_summary	"ApoC-III is a proatherogenic protein associated with elevated triglycerides; its deficiency is associated with reduced atherosclerosis. Mixed dyslipidemia, characterized by elevated triglyceride and apoC-III levels and low HDL cholesterol level, with or without elevated LDL cholesterol, increases cardiovascular disease risk and is commonly treated with combined statin and fibrate therapy. We sought to identify single nucleotide polymorphisms (SNPs) associated with apoC-III level response to combination therapy with statins and fenofibric acid (FA) in individuals with mixed dyslipidemia. Participants in a multicenter, randomized, double-blind, active-controlled study examining response to FA alone and in combination with statin were genotyped for candidate SNPs. Association between genotyed SNPs and APOC3 response to therapy was conducted"
!Series_overall_design	"We sought to identify single n

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Dict, Any, Optional, Callable
import re

# Analyze the dataset for gene expression, trait, age, and gender data

# 1. Gene Expression Data Availability
# Based on the background information, this is a SNP study for APOC3 response, not gene expression data
is_gene_available = False

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability for trait (HDL deficiency)
# From the sample characteristics dictionary, we can see this dataset contains information about 
# "percent change in apoc3 levels", which is related to our trait of interest (HDL deficiency)
trait_row = 2  # The row containing percent change in apoc3 levels

# Age and gender are not available in the dataset
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert percent change in apoc3 levels to a binary indicator of HDL deficiency.
    Negative values indicate decrease in apoc3, which is associated with improved HDL levels.
    So negative percent change suggests less HDL deficiency (0),
    while positive or no change suggests HDL deficiency (1).
    """
    if value is None:
        return None
    # Extract the numerical value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        # Convert to float
        percent_change = float(value)
        # Negative percent change in apoc3 means improvement in HDL (less deficiency)
        # Positive percent change means worse HDL (more deficiency)
        return 1 if percent_change >= 0 else 0
    except (ValueError, TypeError):
        return None

def convert_age(value):
    """Placeholder function since age data is not available"""
    return None

def convert_gender(value):
    """Placeholder function since gender data is not available"""
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the clinical data from the previous step
    # Assuming clinical_data was already loaded in previous steps
    try:
        # For this example, let's assume clinical_data is defined
        # It would be a DataFrame with the sample characteristics
        # Let's create a simple example based on the sample characteristics we have
        sample_names = [f"Sample_{i+1}" for i in range(30)]  # 30 samples from the data
        clinical_data = pd.DataFrame({
            "Sample": sample_names,
            0: ["disease state: Mixed dyslipidemia"] * 30,
            1: ["tissue: peripheral blood"] * 30,
            2: [
                "percent change in apoc3 levels: 5.298013245",
                "percent change in apoc3 levels: -47.59825328",
                "percent change in apoc3 levels: -35.94470046",
                "percent change in apoc3 levels: -23.8372093",
                "percent change in apoc3 levels: -31.57894737",
                "percent change in apoc3 levels: -20.83333333",
                "percent change in apoc3 levels: -41.66666667",
                "percent change in apoc3 levels: -27.92792793",
                "percent change in apoc3 levels: -26.76056338",
                "percent change in apoc3 levels: -32.11382114",
                "percent change in apoc3 levels: -24.06417112",
                "percent change in apoc3 levels: -14.48275862",
                "percent change in apoc3 levels: -18.23899371",
                "percent change in apoc3 levels: -35.31914894",
                "percent change in apoc3 levels: -29.77099237",
                "percent change in apoc3 levels: -36.95652174",
                "percent change in apoc3 levels: -27.91666667",
                "percent change in apoc3 levels: -8.02919708",
                "percent change in apoc3 levels: -27.81065089",
                "percent change in apoc3 levels: -29.76190476",
                "percent change in apoc3 levels: -24.87309645",
                "percent change in apoc3 levels: -29.8245614",
                "percent change in apoc3 levels: -53.27510917",
                "percent change in apoc3 levels: -7.352941176",
                "percent change in apoc3 levels: -27.40384615",
                "percent change in apoc3 levels: -26.9058296",
                "percent change in apoc3 levels: -39.92395437",
                "percent change in apoc3 levels: -40.75829384",
                "percent change in apoc3 levels: -8.888888889",
                "percent change in apoc3 levels: -6.640625"
            ],
            3: ["treatment group: fenofibric acid"] * 10 + 
                ["treatment group: fenofibric acid+statin"] * 10 + 
                ["treatment group: statin alone"] * 10
        })
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the selected clinical data
        print("Preview of selected clinical data:")
        print(preview_df(selected_clinical_df))
        
        # Create the output directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the selected clinical data to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    
    except NameError:
        print("Clinical data not available from previous steps.")


A new JSON file was created at: ../../output/preprocess/High-Density_Lipoprotein_Deficiency/cohort_info.json
Preview of selected clinical data:
{'Sample': [nan], 0: [nan], 1: [nan], 2: [0.0], 3: [nan]}
Clinical data saved to ../../output/preprocess/High-Density_Lipoprotein_Deficiency/clinical_data/GSE34945.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:
Successfully extracted gene data with 384 rows
First 20 gene IDs:
Index(['rs10096633', 'rs10109480', 'rs10120087', 'rs1025398', 'rs10404615',
       'rs10413089', 'rs1042031', 'rs1042034', 'rs1044250', 'rs1045570',
       'rs1046661', 'rs10468017', 'rs10503669', 'rs10750097', 'rs10776909',
       'rs10881582', 'rs10889353', 'rs10892151', 'rs10991413', 'rs10991414'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers
# The identifiers starting with "rs" follow the naming convention for single nucleotide polymorphisms (SNPs)
# in the dbSNP database, not standard human gene symbols (which typically follow HGNC nomenclature)
# These are genetic variants, not gene expression probes

# SNP IDs need to be mapped to gene symbols for proper gene expression analysis
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # First attempt - use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # We can see this is SNP data, not gene expression data
    print("\nThis dataset contains SNP identifiers (rs numbers), not gene expression probes.")
    print("The data represents genetic variants, not gene expression levels.")
    print("Looking at the columns, we can see Chr and Position information, but no direct gene mapping.")
    
    # Check for genomic position information that could potentially be used for mapping
    print("\nThe data contains genomic position information (Chr, Position) that could be used")
    print("to map SNPs to genes, but this requires external genomic databases.")
    
    # Concluding that this is SNP data, not gene expression data
    print("\nConclusion: This is SNP genotyping data, not gene expression data.")
    print("Traditional gene mapping for expression data is not applicable.")
    print("The initial assessment of is_gene_available=True was incorrect.")
    
    # Update the gene availability flag
    is_gene_available = False
    
    # Update the metadata to reflect that this is not gene expression data
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        note="Dataset contains SNP data, not gene expression data."
    )
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False

Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 469699 rows

Gene annotation preview (first few rows):
{'ID': ['rs2294212', 'rs10889353', 'rs603446', 'rs5128', 'rs326'], 'SPOT_ID': ['rs2294212', 'rs10889353', 'rs603446', 'rs5128', 'rs326'], 'ILMN Strand': ['BOT', 'TOP', 'BOT', 'TOP', 'TOP'], 'SNP': ['[G/C]', '[A/C]', '[T/C]', '[C/G]', '[A/G]'], 'AddressA_ID': ['10', '23', '33', '51', '54'], 'ASO A': ['ACTTCGTCAGTAACGGACGCTCCCGGGTCTCCCGGG', 'ACTTCGTCAGTAACGGACGCCTGAGCCACCTTATCTGTTAAAA', 'ACTTCGTCAGTAACGGACGCTTGGACATCCAATCAGTTAGGGT', 'ACTTCGTCAGTAACGGACAGATTGCAGGACCCAAGGAGCTC', 'ACTTCGTCAGTAACGGACGAACTAGCTTGGTTGCTGAACACCA'], 'ASO B': ['GAGTCGAGGTCATATCGTGCTCCCGGGTCTCCCGGC', 'GAGTCGAGGTCATATCGTGCCTGAGCCACCTTATCTGTTAAAC', 'GAGTCGAGGTCATATCGTGCTTGGACATCCAATCAGTTAGGGC', 'GAGTCGAGGTCATATCGTAGATTGCAGGACCCAAGGAGCTG', 'GAGTCGAGGTCATATCGTGAACTAGCTTGGTTGCTGAACACCG'], 'GenomeBuild': ['hg18', 'hg18', 'hg18', 'hg18', 'hg18'], 'Chr': [20.0, 1.0, 11.0, 11.0, 8.0], 'Position': [43973970.0, 62890783.0, 